# Train the model

This section is focus to create a object detection model.

Steps to create and evaluate the  model: 
1. Setup  

    1.1 Setup paths & download pretrained model 
    1.2. Generate Label Map  
    1.3. Generate TensorFlow records  
    1.4. Copy & update pipeline     
    
    
2. Train the Model
3. Evaluate the model


## 1. Setup

* Define the path of the directories and file
* Download TF Object Detection
* Create label Map
* Create TF records
* Update config file for the transfer learning

### 1.1 Setup paths & download pretrained model


Choose the pretrained models from: [Pretrained Model url](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md)


<div class="alert alert-block alert-warning">
<b>Attention:</b> Set model name, model number and labels
</div>


In [ ]:
model_name = 'centernet_resnet50_v7'
model_number = 'model_1'
labels = ['Cola', 'Pepsi']

# Models
MODELS_CONFIG = {
    'model_1': {
        'model_zoo_name': 'centernet_resnet50_v1_fpn_512x512_coco17_tpu-8',
        'model_url': 'http://download.tensorflow.org/models/object_detection/tf2/20200711/centernet_resnet50_v1_fpn_512x512_coco17_tpu-8.tar.gz'
    },
    'model_2': {
        'model_zoo_name': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
        'model_url': 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz'
    }
}

PRETRAINED_MODEL_NAME = MODELS_CONFIG[model_number]['model_zoo_name']
PRETRAINED_MODEL_URL = MODELS_CONFIG[model_number]['model_url']

In [ ]:
import os
import wget

# Paths
paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',model_name), 
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',model_name, 'export'),   
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

# Define files for the scripts & labelmap
files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', model_name, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], 'generate_tfrecord.py'),
    'LABELMAP_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], 'generate_labelmap.py'), 
    'UPDATE_CONFIG_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], 'update_config_file.py'),
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], 'label_map.pbtxt')
}

# Create all the paths from paths dictionary
for path in paths.values():
    if not os.path.exists(path):
        !mkdir {path}
        
# Download and import pretained models
wget.download(PRETRAINED_MODEL_URL)
!move {PRETRAINED_MODEL_NAME+'.tar.gz'} {paths['PRETRAINED_MODEL_PATH']}
!cd {paths['PRETRAINED_MODEL_PATH']} && tar -zxvf {PRETRAINED_MODEL_NAME+'.tar.gz'}

### 1.2. Generate Label Map

<div class="alert alert-block alert-info">
<b>Tip:</b> Classes for labelmap file are added after the -l. Classes must to have the same name as annotations and IDs must to be uniques</div>

In [ ]:
!python {files['LABELMAP_SCRIPT']} -l Cola Pepsi -p {files['LABELMAP']}

### 1.3. Generate TensorFlow records

Now it's time to convert the annotations into TFRecord format. On this step we are converting the .xml files to .record

In [ ]:
# Create the TF records
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'train')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'train.record')} 
!python {files['TF_RECORD_SCRIPT']} -x {os.path.join(paths['IMAGE_PATH'], 'test')} -l {files['LABELMAP']} -o {os.path.join(paths['ANNOTATION_PATH'], 'test.record')} 

### 1.4. Copy & update pipeline
Copy the configuration file to the model folder, and edit configuration file
<div class="alert alert-block alert-warning">
<b>Attention:</b> To change the pipeline open pipeline.config on the model folder. If network architecture change parameter <i>-w</i>
</div>

Choose-w parameters according with architecture:
* ssd
* center_net

In [ ]:
# Copy the pipeline to the model file
!copy {os.path.join(paths['PRETRAINED_MODEL_PATH'], PRETRAINED_MODEL_NAME, 'pipeline.config')} {os.path.join(paths['CHECKPOINT_PATH'])}

# Update the pipeline
!python {files['UPDATE_CONFIG_SCRIPT']} -w center_net -p {files['PIPELINE_CONFIG']} -m {files['LABELMAP']} -t {paths['PRETRAINED_MODEL_PATH']} -n {PRETRAINED_MODEL_NAME} -a {paths['ANNOTATION_PATH']} -c {len(labels)} -b {4}   

## 2. Train the Model

<div class="alert alert-block alert-info">
<b>Tip:</b> To set more parameters, open the configuration file
</div>


In [ ]:
steps = 50000
training_script = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'model_main_tf2.py')
command = "python {} --model_dir={} --pipeline_config_path={} --num_train_steps={}".format(training_script, 
                                                                                             paths['CHECKPOINT_PATH'],
                                                                                             files['PIPELINE_CONFIG'], 
                                                                                             steps)
print(command)

## 3. Evaluate the Model

In [ ]:
command = "python {} --model_dir={} --pipeline_config_path={} --checkpoint_dir={}".format(training_script, 
                                                                                          paths['CHECKPOINT_PATH'],
                                                                                          files['PIPELINE_CONFIG'], 
                                                                                          paths['CHECKPOINT_PATH'])
print(command)

<div class="alert alert-block alert-info">
<b>Tip:</b> Check the evaluations metrics of your model in Tensorboard</div>

In **TensorFlow\workspace\models\** run the command:

<p style="background:black">
<code style="background:black;color:white">tensorboard --logdir=.
</code>
</p>


